In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.decomposition import PCA


import warnings
warnings.filterwarnings(action="ignore")
from utils import target_encoding
from utils import target_encoding_test
import utils
import sklearn


In [2]:
#!pip3 install scikit-learn --upgrade
#Ensure sklearn version is 1.1.2
print(sklearn.__version__)

1.1.2


In [3]:
df_train = pd.read_csv('../data/train_cleaned.csv')
df_test = pd.read_csv('../data/test_cleaned.csv')
df_test.shape
print(df_test.shape)
print(df_train.shape)

(6966, 22)
(20147, 24)


In [4]:
#Proof of 1 to many mapping of planning_area vs subzone
df_train.groupby(['subzone']).planning_area.nunique().unique()
print(np.setdiff1d(df_test.planning_area.unique(), df_train.planning_area.unique()))

#Boon lay is in test data but not in training data , so just replace boon lay in test with jurong east, which is the closest planning_area
df_test['planning_area'].replace('boon lay', 'jurong east', inplace = True)
print(np.setdiff1d(df_test.planning_area.unique(), df_train.planning_area.unique()))



['boon lay']
[]


In [5]:

#Target encoding of planning area


price_avg = df_train.groupby(['planning_area']).price_per_sqft.mean().reset_index(name = 'price_avg_planning_area')
df_train = df_train.merge(price_avg, on=['planning_area'], how = 'left')
df_test = df_test.merge(price_avg, on=['planning_area'], how = 'left')

df_test.shape

(6966, 23)

In [6]:
import lat_long_transform
lat_long_transform.add_price_psf_from_lat_long(df_train, new_column_name='psf_bbox', size = 1000 )
lat_long_transform.add_price_psf_from_lat_long_test(df_train, df_test , new_column_name='psf_bbox', size = 1000 )

df_test.shape

(6966, 24)

In [7]:
df_train = df_train.drop([ 'lat', 'lng'], axis=1)
df_test = df_test.drop([  'lat', 'lng'], axis=1)


In [8]:
#!pip show scikit-learn
#!pip update scikit-learn
#!pip install scikit-learn --upgrade
#Encode age from built_year
df_train['age'] = 2022 - df_train['built_year']
df_test['age'] = 2022 - df_test['built_year']
df_train.describe()

,num_beds,num_baths,size_sqft,price,built_year,pri_sch,sec_sch,mrt_station,population_density,closest_dist_to_mrt,closest_dist_to_pri,closest_dist_to_sec,close_pri_sch,close_sec_sch,closest_dist_to_shop,closest_dist_to_com,price_per_sqft,price_avg_planning_area,psf_bbox,age
count,20147.000000,20147.000000,20147.000000,2.014700e+04,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000,20147.000000
mean,3.117338,2.637365,1725.056336,3.055429e+06,2009.791284,0.958704,0.780811,0.605152,14926.339161,789.993679,741.648893,823.492656,1.793915,1.548767,795.717386,1921.772322,1719.764281,1719.764281,1741.534393,12.208716
std,1.286873,1.464267,2019.252573,4.864724e+06,17.115395,1.295772,0.991577,0.895126,11930.276156,556.011312,537.261735,617.779999,1.547204,1.389442,531.141583,1063.605760,982.283638,753.818810,901.480904,17.115395
min,1.000000,1.000000,65.000000,2.499000e+05,1963.000000,0.000000,0.000000,0.000000,0.269116,5.247005,0.000000,0.003924,0.000000,0.000000,0.000000,89.567740,447.829398,562.151931,440.940000,-6.000000
25%,2.000000,2.000000,807.000000,8.295000e+05,1999.000000,0.000000,0.000000,0.000000,5417.926461,402.621128,369.787867,432.686208,1.000000,0.000000,401.682579,1171.910867,708.395802,1035.970804,921.694853,-1.000000
50%,3.000000,2.000000,1119.000000,1.680000e+06,2016.000000,1.000000,1.000000,0.000000,11059.413028,658.680732,607.254403,685.368100,2.000000,1.000000,680.151057,1680.286557,1657.632743,1541.287388,1632.483841,6.000000
75%,4.000000,3.000000,1528.000000,3.253450e+06,2023.000000,2.000000,1.000000,1.000000,24568.885592,1004.310295,998.306899,1029.828531,3.000000,3.000000,1041.213962,2436.428777,2516.849705,2301.847215,2558.147691,23.000000
max,10.000000,10.000000,86080.000000,1.155000e+08,2028.000000,6.000000,5.000000,4.000000,45396.938412,4119.580609,4031.765090,4462.754211,9.000000,7.000000,4033.626050,5822.497021,4486.890450,3623.018726,16749.637603,59.000000


In [9]:
print(df_test.shape)
print(df_train.shape)

num_cols = [i for i in df_train.columns if df_train[i].dtype in ['int64', 'float64']]
cat_cols = [i for i in df_train.columns if df_train[i].dtype == 'object']

num_cols = [i for i in df_train.columns if df_train[i].dtype in ['int64', 'float64']]
cat_cols = [i for i in df_train.columns if df_train[i].dtype == 'object']

# for col in num_cols:
#     target_encoding(df_train, cat_cols, col)

print(df_test.shape)
print(df_train.shape)

(6966, 23)
(20147, 25)
(6966, 23)
(20147, 25)


In [10]:
num_cols = [i for i in df_train.columns if df_train[i].dtype in ['int64', 'float64']]
cat_cols = [i for i in df_train.columns if df_train[i].dtype == 'object']


print('Number of numerical columns:', len(num_cols))
print(num_cols)
print('Number of categorical columns:', len(cat_cols))
print(cat_cols)


num_cols = [i for i in df_test.columns if df_test[i].dtype in ['int64', 'float64']]
cat_cols = [i for i in df_test.columns if df_test[i].dtype == 'object']


print('Number of numerical columns:', len(num_cols))
print(num_cols)
print('Number of categorical columns:', len(cat_cols))
print(cat_cols)

num_cols.remove('size_sqft')

print(df_test.shape)
print(df_train.shape)

Number of numerical columns: 20
['num_beds', 'num_baths', 'size_sqft', 'price', 'built_year', 'pri_sch', 'sec_sch', 'mrt_station', 'population_density', 'closest_dist_to_mrt', 'closest_dist_to_pri', 'closest_dist_to_sec', 'close_pri_sch', 'close_sec_sch', 'closest_dist_to_shop', 'closest_dist_to_com', 'price_per_sqft', 'price_avg_planning_area', 'psf_bbox', 'age']
Number of categorical columns: 5
['property_type', 'tenure', 'furnishing', 'subzone', 'planning_area']
Number of numerical columns: 18
['built_year', 'num_beds', 'num_baths', 'size_sqft', 'pri_sch', 'sec_sch', 'mrt_station', 'population_density', 'closest_dist_to_mrt', 'closest_dist_to_pri', 'closest_dist_to_sec', 'close_pri_sch', 'close_sec_sch', 'closest_dist_to_shop', 'closest_dist_to_com', 'price_avg_planning_area', 'psf_bbox', 'age']
Number of categorical columns: 5
['property_type', 'tenure', 'furnishing', 'subzone', 'planning_area']
(6966, 23)
(20147, 25)


In [11]:

# from sklearn.preprocessing import StandardScaler
# scale_cols  = ['population_density', 'closest_dist_to_mrt', 'closest_dist_to_pri', 'closest_dist_to_sec', 'closest_dist_to_shop', 'closest_dist_to_com']

# st = StandardScaler()
# st.fit(df_train[scale_cols])
# df_train[scale_cols] = st.transform(df_train[scale_cols])
# df_test[scale_cols] = st.transform(df_test[scale_cols])

#Scaling makes things worse

df_train

,property_type,tenure,num_beds,num_baths,size_sqft,furnishing,subzone,planning_area,price,built_year,...,closest_dist_to_pri,closest_dist_to_sec,close_pri_sch,close_sec_sch,closest_dist_to_shop,closest_dist_to_com,price_per_sqft,price_avg_planning_area,psf_bbox,age
0,hdb,99-year leasehold,3.0,2.0,1115,unspecified,yishun south,yishun,514500.0,1988.0,...,275.582427,183.212612,3,3,621.334350,3338.998737,461.434978,901.355106,611.380924,34.0
1,hdb,99-year leasehold,4.0,2.0,1575,unspecified,serangoon north,serangoon,995400.0,1978.0,...,122.924684,290.839512,3,4,552.543678,2401.881605,632.000000,1722.632056,718.457043,44.0
2,condo,freehold,4.0,6.0,3070,partial,mountbatten,marine parade,8485000.0,2022.0,...,891.475099,894.832576,1,1,824.207598,2171.327898,2763.843648,2301.847215,2221.075608,0.0
3,condo,freehold,3.0,2.0,958,partial,farrer court,bukit timah,2626000.0,2023.0,...,1090.370604,1108.445633,0,0,907.062671,1605.928276,2741.127349,2492.072166,2826.569006,-1.0
4,condo,99-year leasehold,2.0,1.0,732,unspecified,anson,downtown core,1764000.0,2026.0,...,464.834504,1549.017327,1,0,430.415295,1869.573952,2409.836066,2907.235647,2701.046423,-4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20142,condo,99-year leasehold,2.0,2.0,635,unspecified,tagore,ang mo kio,1050000.0,2026.0,...,790.369959,1029.828531,1,0,2014.132610,3163.579338,1653.543307,1441.866157,1997.252541,-4.0
20143,condo,freehold,2.0,2.0,883,unspecified,lavender,kallang,2087400.0,2026.0,...,660.928748,1002.499395,2,0,535.184993,1641.446389,2363.986410,2007.026432,1832.783689,-4.0
20144,condo,freehold,4.0,4.0,1378,unspecified,moulmein,novena,4193700.0,2023.0,...,367.551506,432.686208,2,1,764.409858,856.170432,3043.323657,2294.045637,2780.541618,-1.0
20145,hdb,99-year leasehold,3.0,2.0,1205,unfurnished,woodlands east,woodlands,754800.0,2017.0,...,230.075500,709.923861,3,3,601.719757,2165.768049,626.390041,673.752357,599.859612,5.0


In [12]:
from sklearn.preprocessing import OneHotEncoder
#One hot encoding ['property_type', 'furnishing','planning_area', 'tenure']
encoder = OneHotEncoder(handle_unknown = 'ignore', drop='first', max_categories = 20 , min_frequency = 50)
#encoder = OneHotEncoder(handle_unknown = 'ignore')
#cols = ['property_type', 'furnishing','planning_area', 'tenure']
cols = ['property_type', 'furnishing','tenure']
encoder.fit(df_train[cols]) 
X = encoder.transform(df_train[cols]).toarray()
colnames = []
for col, arr in zip(cols, encoder.get_feature_names_out()):
    #Because we dropped the first category, uncomment line below if we want to dropFirst.
    arr = arr[1:]
    colnames += [col + '_' + x for x in arr]
X = pd.DataFrame(X, columns=encoder.get_feature_names_out(), index=df_train.index).astype(int)
df_train = pd.concat([df_train, X], axis=1).drop(cat_cols, axis = 1)

#test data
X = encoder.transform(df_test[cols]).toarray()
X = pd.DataFrame(X, columns=encoder.get_feature_names_out(), index=df_test.index).astype(int)
df_test = pd.concat([df_test, X], axis=1).drop(cat_cols, axis = 1)
df_test.shape


(6966, 32)

Reducing dimensionality

In [13]:

# X = df_train[num_cols].copy()
# X_test = df_test[num_cols].copy()

# pca = PCA()
# pca.fit(X)

# X_pca = pca.fit_transform(X)

# X_pca_test = pca.transform(X_test)

# component_names = [f'PC{i}' for i in range(X.shape[1])]
# X_pca = pd.DataFrame(X_pca, columns = component_names)
# X_pca_test = pd.DataFrame(X_pca_test, columns = component_names)
# utils.plot_variance(pca)
# plt.show()

'''
df_train = pd.concat(
    [
        df_train,
        pd.DataFrame(
            X, 
            columns=component_names
        )
    ], axis=1
).drop(num_cols, axis = 1)

df_test = pd.concat(
    [
        df_test,
        pd.DataFrame(
            X_test, 
            columns=component_names
        )
    ], axis=1
).drop(num_cols, axis = 1)
'''


'\ndf_train = pd.concat(\n    [\n        df_train,\n        pd.DataFrame(\n            X, \n            columns=component_names\n        )\n    ], axis=1\n).drop(num_cols, axis = 1)\n\ndf_test = pd.concat(\n    [\n        df_test,\n        pd.DataFrame(\n            X_test, \n            columns=component_names\n        )\n    ], axis=1\n).drop(num_cols, axis = 1)\n'

In [14]:
# X_cum = X_pca.drop(columns = X_pca.columns[12:])
# X_cum_test = X_pca_test.drop(columns = X_pca.columns[12:])

# print(X.shape)
# print(X_cum.shape)

# df_train_pca = df_train.drop(columns = num_cols)
# df_test_pca = df_test.drop(columns = num_cols)

# df_test_pca
# df_test.shape


In [15]:
pd.set_option("display.max_columns", None)

df_train.head()

,num_beds,num_baths,size_sqft,price,built_year,pri_sch,sec_sch,mrt_station,population_density,closest_dist_to_mrt,closest_dist_to_pri,closest_dist_to_sec,close_pri_sch,close_sec_sch,closest_dist_to_shop,closest_dist_to_com,price_per_sqft,price_avg_planning_area,psf_bbox,age,property_type_cluster house,property_type_condo,property_type_corner terrace,property_type_executive condo,property_type_hdb,property_type_semi-detached house,property_type_terraced house,property_type_infrequent_sklearn,furnishing_partial,furnishing_unfurnished,furnishing_unspecified,furnishing_infrequent_sklearn,tenure_999-year leasehold,tenure_freehold
0,3.0,2.0,1115,514500.0,1988.0,2.0,2.0,0.0,31517.683928,574.204611,275.582427,183.212612,3,3,621.334350,3338.998737,461.434978,901.355106,611.380924,34.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
1,4.0,2.0,1575,995400.0,1978.0,1.0,0.0,0.0,23280.268731,1734.306081,122.924684,290.839512,3,4,552.543678,2401.881605,632.000000,1722.632056,718.457043,44.0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
2,4.0,6.0,3070,8485000.0,2022.0,0.0,1.0,0.0,5829.779777,1319.767117,891.475099,894.832576,1,1,824.207598,2171.327898,2763.843648,2301.847215,2221.075608,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
3,3.0,2.0,958,2626000.0,2023.0,1.0,1.0,1.0,11059.413028,726.004555,1090.370604,1108.445633,0,0,907.062671,1605.928276,2741.127349,2492.072166,2826.569006,-1.0,0,1,0,0,0,0,0,0,1,0,0,0,0,1
4,2.0,1.0,732,1764000.0,2026.0,0.0,0.0,0.0,775.193798,371.115948,464.834504,1549.017327,1,0,430.415295,1869.573952,2409.836066,2907.235647,2701.046423,-4.0,0,1,0,0,0,0,0,0,0,0,1,0,0,0


In [16]:
df_test.shape

(6966, 32)

In [17]:
df_train.columns.difference(df_test.columns)
df_test.columns.difference(df_train.columns)

Index([], dtype='object')

In [18]:
df_train.to_csv('../data/train_encoded_1.csv', index = False)
df_test.to_csv('../data/test_encoded_1.csv', index = False)

# pca_cum = pd.concat([df_train_pca, X_cum], axis = 1)
# pca_cum_test = pd.concat([df_test_pca, X_cum_test], axis = 1)

# pca_cum.to_csv('../data/train_encoded_pca_cum.csv', index = False)
# pca_cum_test.to_csv('../data/test_encoded_pca_cum.csv', index = False)



# pca = pd.concat([df_train_pca, X_pca], axis = 1)
# pca_test = pd.concat([df_test_pca, X_pca_test], axis = 1)

# pca_cum.to_csv('../data/train_encoded_pca.csv', index = False)
# pca_cum_test.to_csv('../data/test_encoded_pca.csv', index = False)


In [19]:
df_train.shape

(20147, 34)

In [20]:
df_test.shape

(6966, 32)